<a href="https://colab.research.google.com/github/BruceMD/BlockingAlgorithms/blob/main/EM_Algorithm_Playing_Around.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install ipympl

!pip install ipympl

In [141]:
#@title Import Libraries

%matplotlib inline

import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive
from random import randint
from math import sqrt, pi, e, log10, log

In [ ]:
#@title Basic Display of values

x_points = [3, 6, 9, 10, 14, 15, 15.5, 16, 16.5, 17.5]
y_points = [0 for _ in x_points]

plt.plot(x_points, y_points, "ro")
plt.show()


In [ ]:
#@title Interative Graph with two Guassian Distributions

x_points = [3, 6, 9, 10, 14, 15, 15.5, 16, 16.5, 17.5]
y_points = [0 for _ in x_points]


def plot_graph(mu1, sigma1, mu2, sigma2):

    x1 = np.linspace(mu1 - 3*sigma1, mu1 + 3*sigma1, 1000)
    x2 = np.linspace(mu2 - 3*sigma2, mu2 + 3*sigma2, 1000)
    
    plt.plot(x1, stats.norm.pdf(x1, mu1, sigma1))
    plt.plot(x2, stats.norm.pdf(x2, mu2, sigma2))

    plt.plot(x_points, y_points, "ro")
    plt.xlim(0, 20)

    plt.show()


interactive_plot = interactive(plot_graph, mu1=(0.0, 20.0, 0.1), sigma1=(0.0, 8.0, 0.1), mu2=(0.0, 20.0, 0.1), sigma2=(0.0, 8.0, 0.1))

interactive_plot


In [ ]:
#@title Guassian Functions

def probs(x, mu, sigma):
    return (1/(sqrt(2*pi*sigma**2))* e**(-(x - mu)**2/(2*sigma**2)))


def p_dev(x_lis, mu, sigma):
    p_lis = []
    for num in x_lis:
        p_lis.append(probs(num, mu, sigma))

    return p_lis


In [ ]:
#@title Generate list of probabilities for each value x to belong to Gaussian distribution a and b

xa_lis = p_dev(x_points, 3, 0.7)
xb_lis = p_dev(x_points, 3.9, 0.7)

print(xa_lis)
print(xb_lis)

plt.bar(x_points, [log10(y) for y in xa_lis])
plt.bar(x_points, [log10(y) for y in xb_lis])
plt.title("Log10 probability of each point belong to Gaussian Distribution a and b")
plt.show()

In [ ]:
#@title Showing probability of each point to belong to either Gaussian Distribution a or b

def point_prob(x1, x2):
    
    # assuming priors are equal -> P(a) = P(b) = 0.5
    # can recalculate priors later on
    # no idea what priors are but whatevs, can figure it out later

    return (x1*0.5) / (x1*0.5 + x2*0.5)


a_lis = [point_prob(xa_lis[i], xb_lis[i]) for i in range(len(x_points))]
b_lis = [point_prob(xb_lis[i], xa_lis[i]) for i in range(len(x_points))]

print(a_lis)
print(b_lis)

plt.bar(x_points, a_lis)
plt.bar(x_points, b_lis, bottom=a_lis)
#plt.title("Log10 probability of each point belong to Gaussian Distribution a and b")
plt.show()

In [ ]:
#@title Calculate new mu's and sigma's

mu1 = sum([a_lis[i]*x_points[i] for i in range(len(x_points))]) / sum(a_lis)
mu2 = sum([b_lis[i]*x_points[i] for i in range(len(x_points))]) / sum(b_lis)

sigma1 = sqrt(sum([a_lis[i]*(x_points[i] - 3)**2 for i in range(len(x_points))]) / sum(a_lis))
sigma2 = sqrt(sum([b_lis[i]*(x_points[i] - 3.9)**2 for i in range(len(x_points))]) / sum(b_lis))

print(mu1, sigma1)
print(mu2, sigma2)

x1 = np.linspace(mu1 - 3*sigma1, mu1 + 3*sigma1, 1000)
x2 = np.linspace(mu2 - 3*sigma2, mu2 + 3*sigma2, 1000)

plt.plot(x1, stats.norm.pdf(x1, mu1, sigma1))
plt.plot(x2, stats.norm.pdf(x2, mu2, sigma2))

plt.plot(x_points, y_points, "ro")
plt.xlim(0, 20)

plt.show()

This currently shows the first step towards the Expectation Maximisation algorithm, but it does not iterate down to the final values for mu and sigma.

Let's optimise and recursively find the best values

In [ ]:
#@title EM Algorithm Implementation

x_points = [3, 6, 9, 10, 14, 15, 15.5, 16, 16.5, 17.5]

def prob_x(x, mu, sigma):
    return (1/(sqrt(2*pi*sigma**2))* e**(-(x - mu)**2/(2*sigma**2)))


def prob_gaus(x, mu_1, sigma_1, mu_2, sigma_2, p1=0.5, p2=0.5):
    return prob_x(x, mu_1, sigma_1)*p1 / prob_x(x, mu_1, sigma_1)*p1 + prob_x(x, mu_2, sigma_2)*p2


def em_algorithm(mu_a, sigma_a, mu_b, sigma_b):
    
    new_mu_a = sum([(prob_gaus(x, mu_a, sigma_a, mu_b, sigma_b) * x) for x in range(len(x_points))]) / sum([prob_gaus(x, mu_a, sigma_a, mu_b, sigma_b) for x in x_points])

    new_mu_b = sum([(prob_gaus(x, mu_b, sigma_b, mu_a, sigma_a) * x) for x in range(len(x_points))]) / sum([prob_gaus(x, mu_b, sigma_b, mu_a, sigma_a) for x in x_points])

    new_sigma_a = sqrt(sum([prob_gaus(x, mu_a, sigma_a, mu_b, sigma_b) * (mu_a - x)**2 for x in x_points]) / sum([prob_gaus(x, mu_a, sigma_a, mu_b, sigma_b) for x in x_points]))

    new_sigma_b = sqrt(sum([prob_gaus(x, mu_b, sigma_b, mu_a, sigma_a) * (mu_b - x)**2 for x in x_points]) / sum([prob_gaus(x, mu_b, sigma_b, mu_a, sigma_a) for x in x_points]))

    print(new_mu_a, new_sigma_a)
    print(new_mu_b, new_sigma_b)
    print()

    return new_mu_a, new_sigma_a, new_mu_b, new_sigma_b

em_algorithm(3, 0.7, 3.9, 0.7)

m1, s1, m2, s2 = 3, 0.7, 3.9, 0.7

for _ in range(10):
    plot_graph(m1, s1, m2, s2)
    m1, s1, m2, s2 = em_algorithm(m1, s1, m2, s2)

At some point, it would be nice to demonstrate the 3D mapping of optimisation...

In [ ]:
#@title Incomplete Wireframe 3D Graph

fig = plt.figure()
ax = plt.axes(projection='3d')

def f(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))

x = np.linspace(0, 6, 30)
y = np.linspace(0, 6, 30)

X, Y = np.meshgrid(x, y)
Z = f(X, Y)

ax.plot_wireframe(X, Y, Z, cmap='viridis')

plt.title("Wireframe Plot Example")
plt.tight_layout()
plt.show()